# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import os
import cv2
import torch
import time
import subprocess
from datetime import datetime, timedelta
from ultralytics import YOLO
from shutil import move

# Configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SIZE = 640
SEGMENT_DURATION = 600  # 10 minutes in seconds
PROGRESS_INTERVAL = 45  # Seconds between updates
BBOX_PADDING = 0.15
FRAME_SKIP = 1
MIN_SEGMENT_LENGTH = 300


def _get_video_duration(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return frames / fps if fps else 0


def _create_writer(base_name, timestamp, output_dir, segment_num, fps):
   """Create video writer with standardized naming"""
   output_path = os.path.join(
       output_dir,
       f"{base_name}_{timestamp}_part{segment_num:03d}.mp4"
   )
   return cv2.VideoWriter(
       output_path,
       cv2.VideoWriter_fourcc(*'mp4v'),
       fps,
       (MODEL_SIZE, MODEL_SIZE)
   )


class VideoProcessor:
    def __init__(self):
        self.model = YOLO('referee17february.pt').to(DEVICE)
        self.model.fuse()
        if DEVICE == 'cuda':
            self.model.half()
            torch.backends.cudnn.benchmark = True
        self.class_id = self._get_referee_class_id()

    def _get_referee_class_id(self):
        return list(self.model.names.keys())[
            list(self.model.names.values()).index('referee')
        ]

    def process_videos(self, input_dir='dataVideo', output_dir='forLabel', used_dir='used'):
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(used_dir, exist_ok=True)

        for video_file in [f for f in os.listdir(input_dir) if f.endswith('.mp4')]:
            video_path = os.path.join(input_dir, video_file)
            print(f"Starting processing: {video_file}")
            try:
                self._process_single_video(video_path, output_dir, used_dir)
                print(f"Completed processing: {video_file}")
            except Exception as e:
                print(f"Error processing {video_file}: {str(e)}")

    def _process_single_video(self, video_path, output_dir, used_dir):
        if _get_video_duration(video_path) > 3600:
            print("Splitting video into 1-hour chunks...")
            chunks = self._split_into_hourly_chunks(video_path, output_dir)
            print(f"Successfully split into {len(chunks)} chunks")
            
            for chunk in chunks:
                print(f"Processing chunk: {os.path.basename(chunk)}")
                self._process_chunk(chunk, output_dir)
                print(f"Completed chunk: {os.path.basename(chunk)}")
                os.remove(chunk)  # Remove temporary chunk file
            
            self._safe_move(video_path, used_dir)
            print(f"Moved original video to: {used_dir}")
            return

        print("Processing single video chunk...")
        self._process_chunk(video_path, output_dir)
        self._safe_move(video_path, used_dir)
        print(f"Moved original video to: {used_dir}")

    def _process_frame(self, frame, last_valid):
        """Process frame with detection and suppression of YOLO outputs"""
        # Convert to model-compatible format
        h, w = frame.shape[:2]
        new_h = (h // 32) * 32
        new_w = (w // 32) * 32
        resized = cv2.resize(frame, (new_w, new_h))
        
        # Prepare tensor (BCHW format)
        tensor = torch.from_numpy(resized).to(DEVICE)
        tensor = tensor.permute(2, 0, 1)
        tensor = tensor.half() if DEVICE == 'cuda' else tensor.float()
        tensor = (tensor / 255.0).unsqueeze(0)

        # Run inference with suppressed outputs
        with torch.no_grad():
            results = self.model(tensor, verbose=False)[0]

        # Process results with padding
        detected = False
        if len(results.boxes.xyxy) > 0:
            detected = True
            x1, y1, x2, y2 = self._padded_coords(results.boxes.xyxy[0].cpu().numpy(), frame.shape)
            cropped = frame[y1:y2, x1:x2]
            return cv2.resize(cropped, (MODEL_SIZE, MODEL_SIZE)), detected
        return last_valid, detected
    def _padded_coords(self, coords, frame_shape):
        """Calculate padded coordinates with boundary checks"""
        h, w = frame_shape[:2]
        pad_x = int((coords[2]-coords[0])*BBOX_PADDING)
        pad_y = int((coords[3]-coords[1])*BBOX_PADDING)
        return (
            max(0, int(coords[0])-pad_x),
            max(0, int(coords[1])-pad_y),
            min(w, int(coords[2])+pad_x),
            min(h, int(coords[3])+pad_y)
        )
    def _process_chunk(self, video_path, output_dir):
        cap = cv2.VideoCapture(video_path)
        orig_fps = cap.get(cv2.CAP_PROP_FPS)
        
        # Processing setup
        writer = None
        last_valid = None
        segment_num = 1
        segment_start_time = 0
        base_name = os.path.splitext(os.path.basename(video_path))[0]
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        last_progress_update = time.time()
        initial_detection = False
        detections_in_interval = 0

        try:
            while cap.isOpened():
                # Skip frames but maintain temporal accuracy
                for _ in range(FRAME_SKIP):
                    if not cap.grab():
                        break
                
                ret, frame = cap.retrieve()
                if not ret:
                    break

                current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Current time in seconds

                # Process frame and get detection
                processed_frame, detected = self._process_frame(frame, last_valid)
                if detected:
                    detections_in_interval += 1
                
                if processed_frame is not None:
                    if not initial_detection:
                        initial_detection = True
                        segment_start_time = current_time
                        writer = _create_writer(base_name, timestamp, output_dir, segment_num, orig_fps)

                    # Start new segment every 10 minutes
                    if current_time - segment_start_time >= SEGMENT_DURATION:
                        writer.release()
                        segment_num += 1
                        writer = _create_writer(base_name, timestamp, output_dir, segment_num, orig_fps)
                        segment_start_time = current_time

                    # Write processed frame
                    writer.write(processed_frame)
                    last_valid = processed_frame

                # Progress reporting
                if time.time() - last_progress_update >= PROGRESS_INTERVAL:
                    elapsed = timedelta(seconds=int(current_time))
                    pct = (current_time / _get_video_duration(video_path)) * 100
                    print(f"[{elapsed}] {pct:.1f}% processed | Segments: {segment_num} | Detections: {detections_in_interval}")
                    detections_in_interval = 0
                    last_progress_update = time.time()

        finally:
            # Handle final segment
            if writer:
                final_duration = current_time - segment_start_time
                if final_duration >= MIN_SEGMENT_LENGTH:
                    writer.release()
                else:
                    writer.release()
                    os.remove(writer.filename)
            cap.release()

    def _split_into_hourly_chunks(self, video_path, output_dir):
        """Split video into 1-hour chunks with cleanup"""
        base_name = os.path.splitext(os.path.basename(video_path))[0]
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_pattern = os.path.join(output_dir, f"{base_name}_{timestamp}_temp_part%03d.mp4")
        
        try:
            subprocess.run([
                'ffmpeg', '-i', video_path,
                '-c', 'copy',
                '-map', '0',
                '-segment_time', '01:00:00',
                '-f', 'segment',
                '-reset_timestamps', '1',
                '-loglevel', 'error',
                output_pattern
            ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        except subprocess.CalledProcessError as e:
            print(f"Error splitting video: {e.stderr.decode() if e.stderr else 'Unknown error'}")
            raise

        return sorted([
            os.path.join(output_dir, f) 
            for f in os.listdir(output_dir) 
            if f.startswith(f"{base_name}_{timestamp}_temp")
        ])
    
    def _safe_move(self, src, dest_dir):
        """Improved moving with resource cleanup"""
        dest = os.path.join(dest_dir, os.path.basename(src))
        for attempt in range(5):
            try:
                # Ensure no open handles
                if os.path.exists(src):
                    if attempt > 0:
                        time.sleep(2 ** attempt)  # Exponential backoff
                    if os.path.exists(dest):
                        os.remove(dest)
                    move(src, dest)
                    return
            except Exception as e:
                if attempt == 4:
                    print(f"Failed to move file after 5 attempts: {str(e)}")
                    raise
        print(f"Unexpected error moving file: {src}")

    # ... [keep other existing methods] ...

if __name__ == "__main__":
    processor = VideoProcessor()
    print("Starting video processing pipeline...")
    processor.process_videos()
    print("All processing completed!")


YOLO11x summary (fused): 464 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs
Starting video processing pipeline...
Starting processing: videoSplitted.mp4
Processing single video chunk...
[0:00:15] 1.2% processed | Segments: 1 | Detections: 0
[0:00:32] 2.6% processed | Segments: 1 | Detections: 0
[0:00:48] 3.9% processed | Segments: 1 | Detections: 0
[0:01:04] 5.2% processed | Segments: 1 | Detections: 0
[0:01:20] 6.5% processed | Segments: 1 | Detections: 0


KeyboardInterrupt: 